In [16]:
from pymongo import MongoClient
from datetime import datetime
from unidecode import unidecode

import numpy as np
import re

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

In [17]:
def finddoc_byequity(fDb, fCollection_name, fEquity): # acha o documento da empresa pelo ticker da bolsa 
    count = 0
    while count != 1:
        collection = fDb[fCollection_name]
        documents = collection.find({'equity': fEquity})
        count = collection.count_documents({'equity': fEquity})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def upload_infofin(fData_fin, fDoc): # retorna os trimestres da empresa 
    trimestres_upload = []
    for chave, valor in fData_fin.items():
        if isinstance(chave, str) and chave.startswith('2'):
            trimestres_upload.append(chave)
    trimestres_upload = sorted(trimestres_upload, reverse=True)     
    tipo_balanco, trimestres_upload = balance_type(fDoc, trimestres_upload)
    if tipo_balanco == 'ERROR': exit()
    tipo_balanco2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', tipo_balanco)
    return tipo_balanco, tipo_balanco2, trimestres_upload

def balance_type(fEmpresa, fTrimestre): # seleciona o tipo de balanco existente
    tipo_balanco = 'ERROR'   
    # print('\n','entrei aki','\n')
    for key in fEmpresa[fTrimestre[0]]:
        # print(key)
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)       
        # print(key2)
        try:
             pvalor = fEmpresa[fTrimestre[-1]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
            #  print(fTrimestre[-1])
            #  print(pvalor)
             if pvalor != 0: 
                 tipo_balanco = key
                 break
        except KeyError:
            # print('\nCia nao possui DF CONSOLIDADO')
            pass
    novalista_trimestres = []
    if tipo_balanco == 'DFConsolidado':
        for trimestre in fTrimestre:
            # print(fEmpresa[trimestre])
            if len(fEmpresa[trimestre].get(tipo_balanco, {})) != 0:
                    novalista_trimestres.append(trimestre) 
        fTrimestre = novalista_trimestres
    # print(f'\nUtilizando balanço: {tipo_balanco}')        
    return tipo_balanco, fTrimestre

def encontrar_conta(fMydict, fConta):
   f_value = 0 
   for key in fMydict:
      string = unidecode(fMydict[key]['ds_conta'].lower())
      if fConta in string:
         f_value = fMydict[key]['vl_conta']
         break
   return f_value

def transform_Dfp_tri(fDicionario, fConta, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_valor = 0    
    total_tri = fQuarter+3
    f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    if dt.month == 12:  
        if total_tri > fQtdtrimestres:
            f_valor /= 4
        else:
            f_tri_1 = fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_tri_2 = fDicionario[fTrimestres[fQuarter+2]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_tri_3 = fDicionario[fTrimestres[fQuarter+3]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_valor -= (f_tri_1+f_tri_2+f_tri_3) 
    return f_valor

def depreciacao_FDC_tri2(fDicionario, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2): # deletar após testes
    f_valor = 0    
    f_total_tri = fQuarter+1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    if f_total_tri+1 >= fQtdtrimestres:
        try:
            f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
            if f_valor==0:
                try:
                    f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                except KeyError:
                    f_valor = 0   
        except KeyError:
            f_valor = 0
        dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
        if dt.month == 12: f_valor /= 4
        elif dt.month == 9: f_valor /= 3
        elif dt.month == 6: f_valor /= 2
    else:
        if dt.month==3:
            try:
                f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor==0:
                    try:
                        f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor = 0   
            except KeyError:
                f_valor = 0
        else: 
            # trimestre atual
            try:
                f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor==0:
                    try:
                        f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor = 0   
            except KeyError:
                f_valor = 0
            # trimestre anterior
            f_valor_ant = 0
            try:
                f_valor_ant = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor_ant==0:
                    try:
                        f_valor_ant = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor_ant = 0   
            except KeyError:
                f_valor_ant = 0
            f_valor = f_valor - f_valor_ant
    return f_valor

def obter_valor_depreciacao(fDicionario, fTrimestre, fTipobalanco, fTipobalanco2):
    chave = fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)'
    try:
        valor = abs(encontrar_conta(fDicionario[fTrimestre][fTipobalanco][chave], 'depreciacoes'))
        if valor == 0:
            valor = abs(encontrar_conta(fDicionario[fTrimestre][fTipobalanco][chave], 'depreciacao'))
    except KeyError:
        valor = 0
    return valor

def depreciacao_FDC_tri(fDicionario, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_valor = 0    
    f_total_tri = fQuarter + 1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')

    if  dt.month == 3:
        f_valor = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter], fTipobalanco, fTipobalanco2)
    else:
        f_valor_atual = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter], fTipobalanco, fTipobalanco2)
        f_valor_anterior = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter + 1], fTipobalanco, fTipobalanco2)
        f_valor = f_valor_atual - f_valor_anterior       

    if f_total_tri + 1 >= fQtdtrimestres:
        divisor = {'12': 4, '9': 3, '6': 2}.get(str(dt.month), 1)
        f_valor /= divisor

    return f_valor


In [18]:
stock = 'RADL3'

In [19]:
document = finddoc_byequity(db, collection, stock)
doc_stock = document.next() # dicionario referente a stock pick
doc_stock['_id']
tipo_balanco1, tipo_balanco2, trimestres_lista = upload_infofin(doc_stock, doc_stock) 

In [20]:
ebit = []
for i in range(len(trimestres_lista)):
    ebit.append(transform_Dfp_tri(doc_stock, '3.05', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2))
ebit = np.array(ebit)

print(stock)

# calcular 3t, 2t, 1t variacao
variacao_3t = round(ebit[0]/ebit[0+4], 2)
variacao_2t = round(ebit[1]/ebit[1+4], 2) 
variacao_1t = round(ebit[2]/ebit[2+4], 2) 
variacao_21t = round(variacao_2t - variacao_1t, 2)
variacao_32t = round(variacao_3t - variacao_2t, 2)
print(f'Trimestre {trimestres_lista[0]} com variação de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação de {variacao_1t}')
print(f'Variacao do 2 para o 1 = {variacao_21t}')
print(f'Variacao do 3 para o 2 = {variacao_32t}')

# calcular o ebit ltm
variacao_3t = round(np.sum(ebit[0:4])/np.sum(ebit[4:4+4]), 2)
variacao_2t = round(np.sum(ebit[1:1+4])/np.sum(ebit[5:5+4]), 2)
variacao_1t = round(np.sum(ebit[2:2+4])/np.sum(ebit[6:6+4]), 2)
variacao_21ltm = round(variacao_2t - variacao_1t, 2)
variacao_32ltm = round(variacao_3t - variacao_2t, 2)

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {variacao_1t}')
print(f'Variacao do 2 para o 1 ltm = {variacao_21ltm}')
print(f'Variacao do 3 para o 2 ltm = {variacao_32ltm}')

print(f'Total de Ebits {len(ebit)}')
print(f'Total de Trimestres {len(trimestres_lista)}')


RADL3
Trimestre 2023-09-30 com variação de 1.15
Trimestre 2023-06-30 com variação de 1.0
Trimestre 2023-03-31 com variação de 1.57
Variacao do 2 para o 1 = -0.57
Variacao do 3 para o 2 = 0.15
Trimestre 2023-09-30 com variação LTM de 1.23
Trimestre 2023-06-30 com variação LTM de 1.3
Trimestre 2023-03-31 com variação LTM de 1.51
Variacao do 2 para o 1 ltm = -0.21
Variacao do 3 para o 2 ltm = -0.07
Total de Ebits 23
Total de Trimestres 23


In [21]:
ebitda = []
for i in range(len(trimestres_lista)):
    ebit_value = transform_Dfp_tri(doc_stock, '3.05', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2) 
    print(f'trimestre: {trimestres_lista[i]} com ebtida {ebit_value}')
    depreciation_value = depreciacao_FDC_tri(doc_stock, i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2) 
    print(f'trimestre: {trimestres_lista[i]} com depreciação {depreciation_value}')    
    print(f'Ebtida: {ebit_value+depreciation_value}\n')
    ebitda.append(ebit_value+depreciation_value)
ebitda = np.array(ebitda)

print(stock)

# calcular 3t, 2t, 1t variacao
variacao_3t = round(ebitda[0]/ebitda[0+4], 2)
variacao_2t = round(ebitda[1]/ebitda[1+4], 2) 
variacao_1t = round(ebitda[2]/ebitda[2+4], 2) 
variacao_21t = round(variacao_2t - variacao_1t, 2)
variacao_32t = round(variacao_3t - variacao_2t, 2)
print(f'Trimestre {trimestres_lista[0]} com variação de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação de {variacao_1t}')
print(f'Variacao do 2 para o 1 = {variacao_21t}')
print(f'Variacao do 3 para o 2 = {variacao_32t}')

# calcular o ebit ltm
variacao_3t = round(np.sum(ebitda[0:4])/np.sum(ebitda[4:4+4]), 2)
variacao_2t = round(np.sum(ebitda[1:1+4])/np.sum(ebitda[5:5+4]), 2)
variacao_1t = round(np.sum(ebitda[2:2+4])/np.sum(ebitda[6:6+4]), 2)
variacao_21ltm = round(variacao_2t - variacao_1t, 2)
variacao_32ltm = round(variacao_3t - variacao_2t, 2)

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {variacao_1t}')
print(f'Variacao do 2 para o 1 ltm = {variacao_21ltm}')
print(f'Variacao do 3 para o 2 ltm = {variacao_32ltm}')

print(f'Total de Ebitda {len(ebitda)}')
print(f'Total de Trimestres {len(trimestres_lista)}')

trimestre: 2023-09-30 com ebtida 530139.0
funcao // depreciacao: 1282615.0 2023-09-30
funcao // depreciacao: 831517.0 2023-06-30
funcao // depreciacao: 451098.0
trimestre: 2023-09-30 com depreciação 451098.0
Ebtida: 981237.0

trimestre: 2023-06-30 com ebtida 642979.0
funcao // depreciacao: 831517.0 2023-06-30
funcao // depreciacao: 406557.0 2023-03-31
funcao // depreciacao: 424960.0
trimestre: 2023-06-30 com depreciação 424960.0
Ebtida: 1067939.0

trimestre: 2023-03-31 com ebtida 432683.0
trimestre: 2023-03-31 com depreciação 406557.0
Ebtida: 839240.0

trimestre: 2022-12-31 com ebtida 458834.0
funcao // depreciacao: 1515538.0 2022-12-31
funcao // depreciacao: 1117120.0 2022-09-30
funcao // depreciacao: 398418.0
trimestre: 2022-12-31 com depreciação 398418.0
Ebtida: 857252.0

trimestre: 2022-09-30 com ebtida 459088.0
funcao // depreciacao: 1117120.0 2022-09-30
funcao // depreciacao: 735896.0 2022-06-30
funcao // depreciacao: 381224.0
trimestre: 2022-09-30 com depreciação 381224.0
Ebtida

In [22]:
# RADL3
# Trimestre 2023-09-30 com variação de 1.17
# Trimestre 2023-06-30 com variação de 1.05
# Trimestre 2023-03-31 com variação de 1.31
# Variacao do 2 para o 1 = -0.26
# Variacao do 3 para o 2 = 0.12
# Trimestre 2023-09-30 com variação LTM de 1.19
# Trimestre 2023-06-30 com variação LTM de 1.21
# Trimestre 2023-03-31 com variação LTM de 1.31
# Variacao do 2 para o 1 ltm = -0.1
# Variacao do 3 para o 2 ltm = -0.02
# Total de Ebits 23
# Total de Trimestres 23